In [1]:
import os
import time

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

tf.distribute.OneDeviceStrategy(device="/gpu:0")

In [2]:
# Even with large batch sizes, there isn't much to speed up with mixed_float due to training inputs batch data. Slows down training on older GPUs

# policy = tf.keras.mixed_precision.Policy("mixed_float16")
# tf.keras.mixed_precision.experimental.set_policy(policy)
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
path_to_file = tf.keras.utils.get_file(
    "shakespeare.txt",
    "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt",
)

In [4]:
text = open(path_to_file, "rb").read().decode(encoding="utf-8")
print(f"Length of test: {len(text)} characters")

Length of test: 1115394 characters


In [5]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
vocab = sorted(set(text))
print(vocab)
print(f"{len(vocab)} unique characters")

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65 unique characters


In [7]:
example_texts = ["abcdefg", "xyz"]

chars = tf.strings.unicode_split(example_texts, input_encoding="UTF-8")
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [8]:
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab))

In [9]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[41, 42, 43, 44, 45, 46, 47], [64, 65, 66]]>

In [10]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True
)

In [11]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [12]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [13]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [14]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, "UTF-8"))
all_ids.numpy()

array([20, 49, 58, ..., 47, 10,  2], dtype=int64)

In [15]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [16]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode("utf-8"))

F
i
r
s
t
 
C
i
t
i


In [17]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

In [18]:
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [19]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [20]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [21]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [22]:
dataset = sequences.map(split_input_target)

In [23]:
for input_example, target_example in dataset.take(1):
    print("Input Text:", text_from_ids(input_example).numpy())
    print("Input ID's:", input_example)
    print("Target Text:", text_from_ids(target_example).numpy())
    print("Target ID's:", target_example)

Input Text: b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Input ID's: tf.Tensor(
[20 49 58 59 60  3 17 49 60 49 66 45 54 12  2 16 45 46 55 58 45  3 63 45
  3 56 58 55 43 45 45 44  3 41 54 65  3 46 61 58 60 48 45 58  8  3 48 45
 41 58  3 53 45  3 59 56 45 41 51 10  2  2 15 52 52 12  2 33 56 45 41 51
  8  3 59 56 45 41 51 10  2  2 20 49 58 59 60  3 17 49 60 49 66 45 54 12
  2 39 55 61], shape=(100,), dtype=int64)
Target Text: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
Target ID's: tf.Tensor(
[49 58 59 60  3 17 49 60 49 66 45 54 12  2 16 45 46 55 58 45  3 63 45  3
 56 58 55 43 45 45 44  3 41 54 65  3 46 61 58 60 48 45 58  8  3 48 45 41
 58  3 53 45  3 59 56 45 41 51 10  2  2 15 52 52 12  2 33 56 45 41 51  8
  3 59 56 45 41 51 10  2  2 20 49 58 59 60  3 17 49 60 49 66 45 54 12  2
 39 55 61  3], shape=(100,), dtype=int64)


In [24]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset.shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [25]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [26]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(
            units=rnn_units, return_sequences=True, return_state=True
        )
#         self.lstm2 = tf.keras.layers.LSTM(
#             units=rnn_units, return_sequences=True, return_state=True
#         )
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)

        if states is None:
            states = self.lstm.get_initial_state(x)
        x, states_h, states_c = self.lstm(x, initial_state=states, training=training)
        states = [states_h, states_c]
        
#         x, states_h, states_c = self.lstm2(x, initial_state=states, training=training)
#         states = [states_h, states_c]
        
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [27]:
model = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
)

In [28]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(
        example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)"
    )

(64, 100, 67) # (batch_size, sequence_length, vocab_size)


In [29]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  17152     
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
dense (Dense)                multiple                  68675     
Total params: 5,332,803
Trainable params: 5,332,803
Non-trainable params: 0
_________________________________________________________________


In [30]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [31]:
sampled_indices

array([17, 16,  5, 60, 23, 50, 10, 22, 52, 58, 61, 24,  4, 43, 19,  8,  2,
        5, 28, 22, 19, 64, 16, 34, 14, 60, 64, 13, 11, 51, 48, 65, 58,  9,
       61, 32, 14, 28,  0, 49, 12, 56, 16,  0, 56, 29, 22, 56, 38, 11, 34,
       26, 12, 39, 66, 13, 21, 21, 10, 43, 16, 18, 56, 39, 47, 41, 65, 54,
       39, 19,  2,  6,  2, 52, 40, 65,  8, 13,  9, 18, 53, 60, 24, 57, 10,
        3, 56, 40, 37, 50, 30, 24, 56, 19, 63, 19, 65, 24, 17,  6],
      dtype=int64)

In [32]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'here great Aufidius lies: is he in Antium?\n\nCitizen:\nHe is, and feasts the nobles of the state\nAt hi'

Next Char Predictions:
 b'CB$tIj.HlruJ!cE,\n$NHExBT?tx;3khyr-uR?Ni:pBpOHpX3TL:Yz;GG.cBDpYgaynYE\n&\nlZy,;-DmtJq. pZWjPJpEwEyJC&'


In [33]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [34]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print(
    "Prediction shape: ",
    example_batch_predictions.shape,
    " # (batch_size, sequence_length, vocab_size)",
)
print("Mean loss:", mean_loss)

Prediction shape:  (64, 100, 67)  # (batch_size, sequence_length, vocab_size)
Mean loss: 4.2056117


In [35]:
# Newly created models shouldn't have weights created that are too certain in it's result without training, the exponent of the mean loss should be close to the number of inputs
print(len(vocab))
print(tf.exp(mean_loss).numpy())

65
67.06161


In [36]:
model.compile(optimizer="adam", loss=loss)

In [37]:
# checkpoints directory
checkpoint_dir = "./training_checkpoints"

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix, save_weights_only=True
)

In [38]:
EPOCHS = 30

# early stopping is cool but since there isn't a validation it doesn't quite work
# earlystopping = tf.keras.callbacks.EarlyStopping(
#     monitor="loss", mode="auto", patience=3, restore_best_weights=True
# )

In [39]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
172/172 [==============================] - 20s 97ms/step - loss: 3.2614
Epoch 2/30
172/172 [==============================] - 18s 97ms/step - loss: 2.1544
Epoch 3/30
172/172 [==============================] - 18s 98ms/step - loss: 1.8518
Epoch 4/30
172/172 [==============================] - 18s 98ms/step - loss: 1.6598
Epoch 5/30
172/172 [==============================] - 18s 98ms/step - loss: 1.5357
Epoch 6/30
172/172 [==============================] - 18s 98ms/step - loss: 1.4472
Epoch 7/30
172/172 [==============================] - 18s 98ms/step - loss: 1.3850
Epoch 8/30
172/172 [==============================] - 18s 98ms/step - loss: 1.3362
Epoch 9/30
172/172 [==============================] - 18s 98ms/step - loss: 1.2856
Epoch 10/30
172/172 [==============================] - 18s 98ms/step - loss: 1.2536
Epoch 11/30
172/172 [==============================] - 18s 98ms/step - loss: 1.2177
Epoch 12/30
172/172 [==============================] - 18s 98ms/step - loss: 1.1840
E

In [40]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent '' or [UNK] from being Generated
        skip_ids = self.ids_from_chars(["", "[UNK]"])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put an -inf at each bad index
            values=[-float("inf")] * len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())],
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, "UTF-8")
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(
            inputs=input_ids, states=states, return_state=True
        )

        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits / self.temperature
        # Apply the prediction mask: prevent "" or [UNK] from being generated
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and the model state.
        return predicted_chars, states

In [41]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [42]:
start = time.time()
states = None
next_char = tf.constant(["ROMEO:"])
result = [next_char]

for n in range(2000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode("utf-8"), "\n\n" + "_" * 80)
print("\nRun time:", end - start)

ROMEO:
One of that honour.
If thou respect them for their lives like chast.
Need thou Romes, Marcius! O, how cares lived, pray,
That virestantfur with the desting of my joy
But what you do, so well abise; for mill,
Who greshall I make a body of a king:
itherifference without mildness, present
the medely week: if you threw busy-day,
What crown his state, for all the lusty gifts
have rock'd upon this foot-paped with our compossion.

BIANCA:
Why, thus's growing she?

ROMEO:
Thou setter already.

First Servingman:
Why, here's my daughter: what a trunch, bast are be?
I'll still be burnt by not.

MIRANDA:
Tell me, gentle comm; come to my cause;
Where lessours shall go embrace me the matter
Than he be pitied much: let 'tis be:
I am a goodle's eyes and alter it,
Or it conclude their mother in this hour
And uped scalue to the Tower. Thou hast no carely
Would it were the struct to prove a sweet wife then.
How should they, since that I have more care
To give the life be slain, whence 'gainst me!
